

---



In [ ]:
#@title Setup
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import getpass
import smtplib
from email.message import EmailMessage
from google.colab import files
import io
import pandas as pd

In [ ]:
#@title Credenciales del correo USFQ
EMAIL_ADDRESS = input("Tu correo USFQ: ")
print('Tu contraseña del correo')
EMAIL_PASSWORD = getpass.getpass()

In [ ]:
#@title Sube la tabla de excel con los destinatarios de correo, email debe ser la segunda columna
# Columnas
# userID  email title firstname lastname  keep  optout

print("\n----")
print("Adjunta el excel con la tabla de recipientes \n")
uploaded = files.upload()
filename=[key for key in uploaded.keys()][0]
df = pd.read_excel(io.BytesIO(uploaded[filename]))

In [ ]:
df

# Plantillas HTML

In [ ]:
# Descargar plantillas

!rm -rf UPDATE_BIGSURV23.html
!rm -rf CALL_BIGSURV23.html
!rm -rf SPANISH_BIGSURV23.html
!wget -nv https://raw.githubusercontent.com/mayait/USFQ_BigSurv/main/newsleter_march_23/CALL_BIGSURV23.html
!wget -nv https://raw.githubusercontent.com/mayait/USFQ_BigSurv/main/newsleter_march_23/UPDATE_BIGSURV23.html
!wget -nv https://raw.githubusercontent.com/mayait/USFQ_BigSurv/main/newsleter_march_23/SPANISH_BIGSURV23.html

# Cargar Style
style = ''' 
<style type="text/css">
body,table,td{font-family:Helvetica,Arial,sans-serif !important}.ExternalClass{width:100%}.ExternalClass,.ExternalClass p,.ExternalClass span,.ExternalClass font,.ExternalClass td,.ExternalClass div{line-height:150%}a{text-decoration:none}*{color:inherit}a[x-apple-data-detectors],u+#body a,#MessageViewBody a{color:inherit;text-decoration:none;font-size:inherit;font-family:inherit;font-weight:inherit;line-height:inherit}img{-ms-interpolation-mode:bicubic}table:not([class^=s-]){font-family:Helvetica,Arial,sans-serif;mso-table-lspace:0pt;mso-table-rspace:0pt;border-spacing:0px;border-collapse:collapse}table:not([class^=s-]) td{border-spacing:0px;border-collapse:collapse}@media screen and (max-width: 600px){.w-full,.w-full>tbody>tr>td{width:100% !important}.p-3:not(table),.p-3:not(.btn)>tbody>tr>td,.p-3.btn td a{padding:12px !important}*[class*=s-lg-]>tbody>tr>td{font-size:0 !important;line-height:0 !important;height:0 !important}.s-3>tbody>tr>td{font-size:12px !important;line-height:12px !important;height:12px !important}.s-5>tbody>tr>td{font-size:20px !important;line-height:20px !important;height:20px !important}.s-10>tbody>tr>td{font-size:40px !important;line-height:40px !important;height:40px !important}}
</style>
'''

import re

# Abrir el HTML
with open('UPDATE_BIGSURV23.html', 'r') as file:
    UPDATE_BIGSURV23 = file.read()
# Reemplazar todo lo que está en style.
UPDATE_BIGSURV23 = re.sub(r'<style(.*?)</style>', "{style}", UPDATE_BIGSURV23, flags=re.DOTALL)

# Abrir el HTML
with open('SPANISH_BIGSURV23.html', 'r') as file:
    SPANISH_BIGSURV23 = file.read()
# Reemplazar todo lo que está en style.
SPANISH_BIGSURV23 = re.sub(r'<style(.*?)</style>', "{style}", SPANISH_BIGSURV23, flags=re.DOTALL)

# Abrir el HTML
with open('CALL_BIGSURV23.html', 'r') as file:
    CALL_BIGSURV23 = file.read()
# Reemplazar todo lo que está en style.
CALL_BIGSURV23 = re.sub(r'<style(.*?)</style>', "{style}", CALL_BIGSURV23, flags=re.DOTALL)


# **Subjects and template**

In [ ]:
body = UPDATE_BIGSURV23
subject = "UPDATE ON PUBLICATION INFORMATION FOR BIGSURV23"

#body = CALL_BIGSURV23
#subject = "BigSurv23: Call for Abstracts"

#body = SPANISH_BIGSURV23 
#subject = "BigSurv23: Solicitud de Abstracts" 

# Preview
from IPython.display import HTML
HTML(body.format(style=style))

In [ ]:
#@title Función enviar correo

def send_mail(mail_to, firstname, keep, optout, body):
  msg = EmailMessage()
  msg['Subject'] = subject
  msg['From'] = EMAIL_ADDRESS 
  msg['To'] = mail_to 


  body = body.format(firstname=firstname, 
                     style=style,
                     keep=keep,
                     optout=optout)
  msg.set_content(body, subtype='html')


  connection = smtplib.SMTP(host='smtp.office365.com', port=587)
  connection.starttls()
  connection.login(EMAIL_ADDRESS,EMAIL_PASSWORD)
  connection.send_message(msg)
  connection.quit()

In [ ]:
#@title Enviar correo { display-mode: "form" }

for row in df.iterrows():
  send_mail(mail_to = row[1][1], 
            firstname = '', 
            keep=1, 
            optout=1, 
            body=body)
  print(row[0],row[1][1], "OK")

In [ ]:
# Test
send_mail(mail_to = 'carlapalau88@gmail.com', firstname = '', keep=1, optout=1, body=body)